In [1]:
#import dependencies
from config import api_key
import pandas as pd
#from sodapy import Socrata
from ipywidgets import interact, interactive
import ipywidgets as widgets
#from ipyleaflet import Map
import requests 
#import matplotlib.pyplot as plt
#import matplotlib.cm

#from mpl_toolkits.basemap import Basemap
#from mpl_toolkits.basemap import Basemap
#from matplotlib.patches import Polygon
#from matplotlib.collections import PatchCollection
#from matplotlib.colors import Normalize

#client = Socrata('data.transportation.gov', api_key)



#results = client.get('38pg-dj5d', city1='Austin, TX')

In [25]:
"""
Function that gets a unique list of cities 

@param1: column: the name of the column

returns: a list of unique cities
"""
def get_cities(column):
    #return pd.read_csv('./resources/data/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv')[column].sort_values().unique()
    return pd.read_csv(f'./resources/data/{column}.csv')[column]

In [26]:
get_cities('city1')

0                                 Albany, NY
1                            Albuquerque, NM
2             Allentown/Bethlehem/Easton, PA
3                               Amarillo, TX
4                              Asheville, NC
5                                  Aspen, CO
6            Atlanta, GA (Metropolitan Area)
7                          Atlantic City, NJ
8                                Augusta, GA
9                                 Austin, TX
10                                Bangor, ME
11                           Baton Rouge, LA
12                            Bellingham, WA
13                          Bend/Redmond, OR
14                              Billings, MT
15                            Birmingham, AL
16                       Bismarck/Mandan, ND
17                    Bloomington/Normal, IL
18                                 Boise, ID
19            Boston, MA (Metropolitan Area)
20                               Bozeman, MT
21                               Buffalo, NY
22        

In [3]:
#save the data to a csv since the original dataset it too large to save on github
#pd.DataFrame(get_cities('city2')).rename(columns={0: 'city2'}).to_csv('./resources/data/city2.csv', index=False)

In [4]:
"""
Function that builds a dataframe based on input param values

@param1: year
@param2: city1: Typically Austin, TX
@param3: data_request_type
         0: for all records
         1: Average fare and total passengers by year
         2: List of carriers for the year
         3: List of segments for the year
"""
def build_df(year, city1, data_request_type):#, city2):
    try:
        base_url = f'https://data.transportation.gov/resource/38pg-dj5d.json?$$app_token={api_key}'
        params = {'city1': f'{city1}'
                  , 'year': f'{year}'
                  #, 'city2': f'{city2}'
                 }

        results = requests.get(base_url, params).json()
        df = pd.DataFrame(results)
        df['fare'] = pd.to_numeric(df['fare'])
        df['passengers'] = pd.to_numeric(df['passengers'])
        df['quarter'] = pd.to_numeric(df['quarter'])
        df['year'] = pd.to_numeric(df['year'])
        
        df1 = df[['carrier_lg'
                   #, 'carrier_low'
                   , 'city1'
                   , 'city2'
                   , 'location_2'
                   , 'fare'
                   , 'passengers'
                   , 'quarter'
                   , 'year']].sort_values(['city2', 'year', 'quarter'])
        if data_request_type == 0:
            #All records
            return df1
        if data_request_type == 1:
            #Average Fare and Total Passengers by Year
            return df1.groupby(['year', 'city1', 'city2']).agg({'fare': 'mean', 'passengers': 'sum'}).reset_index()            
        if data_request_type == 2:
            #Unique list of carriers for the Year
            return pd.DataFrame(df['carrier_lg'].sort_values().unique()).rename(columns={0: 'carrier_lg'})            
        if data_request_type == 3:
            #List of segments for the year
            return pd.DataFrame(df['city2'].sort_values().unique()).rename(columns={0: 'city2'})
    except KeyError:
        print("Looks like there is no data for this city pair, try another")

In [27]:
interact(build_df, year=(2007, 2017), city1=get_cities('city1')
         , data_request_type={'All records': 0
                              , 'Average fare and total passengers by year': 1
                              , 'List of carriers for the year': 2
                              , 'List of segments for the year': 3
                             })


interactive(children=(IntSlider(value=2012, description='year', max=2017, min=2007), Dropdown(description='cit…

<function __main__.build_df(year, city1, data_request_type)>